In [75]:
import pandas as pd
import numpy as np
import csv as csv

In [76]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [77]:
#Shuffle the datasets
from sklearn.utils import shuffle

In [78]:
#Learning curve
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [79]:
import seaborn as sns

In [80]:
addpoly = True
plot_lc = 0   # 1--display learning curve/ 0 -- don't display

In [81]:
train_dataset=pd.read_csv("D:/Projects/Kaggle/titanic/train.csv")
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [82]:
test_dataset=pd.read_csv("D:/Projects/Kaggle/titanic/test.csv")
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [83]:
train_dataset.shape

(891, 12)

In [84]:
test_dataset.shape

(418, 11)

###### DATASET VISUALISATION

In [85]:
print("All IDs are unique") if train_dataset.PassengerId.nunique()==train_dataset.shape[0] else print("Oops! Not unique")
#nunique gives the count of unique elements in an array

All IDs are unique


In [86]:
h=np.intersect1d(train_dataset.PassengerId.values,test_dataset.PassengerId.values)
#return an array of common,unique elements

In [87]:
if len(h)==0:
    print("Train and test are unique")
else:
    print("The train nd test are not unique")

Train and test are unique


In [88]:
datasethasNan=False
if train_dataset.count().min() == train_dataset.shape[0] and test_dataset.count().min() == test_dataset.shape[0] :
    print('We do not need to worry about missing values.') 
else:
    datasethasNan = True
    print('oops we have NAN')

oops we have NAN


In [89]:
print('----train dataset column types information-------')
dtype_df = train_dataset.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

----train dataset column types information-------


,Column Type,Count
0,int64,5
1,float64,2
2,object,5


In [90]:
if datasethasNan==True:
    nas=pd.concat([train_dataset.isnull().sum(),test_dataset.isnull().sum()],axis=1, keys=['Train Dataset','Test Dataset'])
    print('---Nan in the datasets---')
    print(nas[nas.sum(axis=1)>0])

---Nan in the datasets---
          Train Dataset  Test Dataset
Age                 177          86.0
Cabin               687         327.0
Embarked              2           0.0
Fare                  0           1.0


C:\Users\harsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [91]:
#Class vs Survived
print(train_dataset[['Pclass','Survived']].groupby(['Pclass'],as_index=False).mean().sort_values(by="Survived",ascending=False))

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [92]:
#Sex vs Survived
print(train_dataset[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',ascending=False))

      Sex  Survived
0  female  0.742038
1    male  0.188908


In [93]:
#SibSp vs Survived
print(train_dataset[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False))

   SibSp  Survived
1      1  0.535885
2      2  0.464286
0      0  0.345395
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000


In [94]:
#Parch vs Survived
print(train_dataset[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False))

   Parch  Survived
3      3  0.600000
1      1  0.550847
2      2  0.500000
0      0  0.343658
5      5  0.200000
4      4  0.000000
6      6  0.000000


###### START DATA PRE-PROCESSING

In [95]:
#Start Data Cleaning

#Age
train_random_ages = np.random.randint(train_dataset["Age"].mean() - train_dataset["Age"].std(),
                                          train_dataset["Age"].mean() + train_dataset["Age"].std(),
                                          size = train_dataset["Age"].isnull().sum())

#print(train_random_ages)
test_random_ages = np.random.randint(test_dataset["Age"].mean() - test_dataset["Age"].std(),
                                          test_dataset["Age"].mean() + test_dataset["Age"].std(),
                                          size = test_dataset["Age"].isnull().sum())
train_dataset["Age"][np.isnan(train_dataset["Age"])] = train_random_ages
test_dataset["Age"][np.isnan(test_dataset["Age"])]=test_random_ages
#remove any decimal value in the array
train_dataset['Age'] = train_dataset['Age'].astype(int) 
test_dataset["Age"]=test_dataset['Age'].astype(int)


C:\Users\harsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\harsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [96]:
# Embarked 
train_dataset["Embarked"].fillna('S', inplace=True)
test_dataset["Embarked"].fillna('S', inplace=True)
train_dataset['Port'] = train_dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_dataset['Port'] = test_dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
del train_dataset["Embarked"]
del test_dataset["Embarked"]

# Fare
test_dataset["Fare"].fillna(test_dataset["Fare"].median(), inplace=True)

In [97]:
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,0


In [98]:
#for specifying a cabin/No Cabin
train_dataset["Has_Cabin"]=train_dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test_dataset['Has_Cabin'] = test_dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#print(train_dataset.head(20))

In [99]:
#New feature extraction
full_dataset=[train_dataset,test_dataset]
for dataset in full_dataset:
    dataset["Fam_Size"]=dataset['SibSp']+1+dataset['Parch']
    
for dataset in full_dataset:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['Fam_Size'] == 1, 'IsAlone'] = 1
    
print(dataset.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

   Age  SibSp  Parch   Ticket     Fare Cabin  Port  Has_Cabin  Fam_Size  \
0   34      0      0   330911   7.8292   NaN     2          0         1   
1   47      1      0   363272   7.0000   NaN     0          0         2   
2   62      0      0   240276   9.6875   NaN     2          0         1   
3   27      0      0   315154   8.6625   NaN     0          0         1   
4   22      1      1  3101298  12.2875   NaN     0          0         3   

   IsAlone  
0        1  
1        0  
2        1  
3        1  
4  

In [100]:
#Name Tagging
train_dataset['Title'] = train_dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
test_dataset['Title'] = test_dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in full_dataset:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle','Miss')
    dataset['Title']=dataset['Title'].replace('Ms','Miss')
    dataset['Title']=dataset['Title'].replace('Mme','Mrs')
    
print(dataset.head(20))

    PassengerId  Pclass                                               Name  \
0           892       3                                   Kelly, Mr. James   
1           893       3                   Wilkes, Mrs. James (Ellen Needs)   
2           894       2                          Myles, Mr. Thomas Francis   
3           895       3                                   Wirz, Mr. Albert   
4           896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5           897       3                         Svensson, Mr. Johan Cervin   
6           898       3                               Connolly, Miss. Kate   
7           899       2                       Caldwell, Mr. Albert Francis   
8           900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9           901       3                            Davies, Mr. John Samuel   
10          902       3                                   Ilieff, Mr. Ylio   
11          903       1                         Jones, Mr. Charl

In [101]:
#assign Family Size

for dataset in full_dataset:
    dataset['FamilySizeGroup'] = 'Small'
    dataset.loc[dataset['Fam_Size'] == 1, 'FamilySizeGroup'] = 'Alone'
    dataset.loc[dataset['Fam_Size'] >= 5, 'FamilySizeGroup'] = 'Big'
    
print(dataset.head(10))

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   
5          897       3                    Svensson, Mr. Johan Cervin    male   
6          898       3                          Connolly, Miss. Kate  female   
7          899       2                  Caldwell, Mr. Albert Francis    male   
8          900       3     Abrahim, Mrs. Joseph (Sophie Halaut Easu)  female   
9          901       3                       Davies, Mr. John Samuel    male   

   Age  SibSp  Parch     Ticket     Fare Cabin  Port  Has_Cabin  Fam_Size  \
0   34      0      0     330911   7.8292  

In [102]:
train_dataset[['Fam_Size', 'Survived']].groupby(['Fam_Size'], as_index=False).mean()

,Fam_Size,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


In [103]:
for dataset in full_dataset:
    dataset['Sex']=dataset['Sex'].map( {'female':1, 'male':0}).astype(int)

In [104]:
for dataset in full_dataset:    
    dataset.loc[ dataset['Age'] <= 14, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 14) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
print(dataset.head())

   PassengerId  Pclass                                          Name  Sex  \
0          892       3                              Kelly, Mr. James    0   
1          893       3              Wilkes, Mrs. James (Ellen Needs)    1   
2          894       2                     Myles, Mr. Thomas Francis    0   
3          895       3                              Wirz, Mr. Albert    0   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)    1   

   Age  SibSp  Parch   Ticket     Fare Cabin  Port  Has_Cabin  Fam_Size  \
0    2      0      0   330911   7.8292   NaN     2          0         1   
1    2      1      0   363272   7.0000   NaN     0          0         2   
2    3      0      0   240276   9.6875   NaN     2          0         1   
3    1      0      0   315154   8.6625   NaN     0          0         1   
4    1      1      1  3101298  12.2875   NaN     0          0         3   

   IsAlone Title FamilySizeGroup  
0        1    Mr           Alone  
1        0   Mrs

In [105]:
for dataset in full_dataset:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
dataset['Fare'].median()

2.0

In [106]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
family_mapping = {"Small": 0, "Alone": 1, "Big": 2}
for dataset in full_dataset:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['FamilySizeGroup'] = dataset['FamilySizeGroup'].map(family_mapping)

In [107]:
for dataset in full_dataset:
    dataset['IsChildandRich'] = 0
    dataset.loc[(dataset['Age'] <= 0) & (dataset['Pclass'] == 1 ),'IsChildandRich'] = 1  
    dataset.loc[(dataset['Age'] <= 0) & (dataset['Pclass'] == 2 ),'IsChildandRich'] = 1 
print(dataset.head(20))

    PassengerId  Pclass                                               Name  \
0           892       3                                   Kelly, Mr. James   
1           893       3                   Wilkes, Mrs. James (Ellen Needs)   
2           894       2                          Myles, Mr. Thomas Francis   
3           895       3                                   Wirz, Mr. Albert   
4           896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5           897       3                         Svensson, Mr. Johan Cervin   
6           898       3                               Connolly, Miss. Kate   
7           899       2                       Caldwell, Mr. Albert Francis   
8           900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9           901       3                            Davies, Mr. John Samuel   
10          902       3                                   Ilieff, Mr. Ylio   
11          903       1                         Jones, Mr. Charl

In [108]:
for data in full_dataset:
    # classify Cabin by fare
    data['Cabin'] = data['Cabin'].fillna('X')
    data['Cabin'] = data['Cabin'].apply(lambda x: str(x)[0])
    data['Cabin'] = data['Cabin'].replace(['A', 'D', 'E', 'T'], 'M')
    data['Cabin'] = data['Cabin'].replace(['B', 'C'], 'H')
    data['Cabin'] = data['Cabin'].replace(['F', 'G'], 'L')
    data['Cabin'] = data['Cabin'].map({'X': 0, 'L': 1, 'M': 2, 'H': 3}).astype(int)

In [109]:
del train_dataset['Name']
del test_dataset['Name']

del train_dataset['SibSp']
del test_dataset['SibSp']

del train_dataset['Parch']
del test_dataset['Parch']

del train_dataset['Fam_Size']
del test_dataset['Fam_Size']

In [110]:
dataset.head(20)

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin,Port,Has_Cabin,IsAlone,Title,FamilySizeGroup,IsChildandRich
0,892,3,0,2,330911,0,0,2,0,1,1,1,0
1,893,3,1,2,363272,0,0,0,0,0,3,0,0
2,894,2,0,3,240276,1,0,2,0,1,1,1,0
3,895,3,0,1,315154,1,0,0,0,1,1,1,0
4,896,3,1,1,3101298,1,0,0,0,0,3,0,0
5,897,3,0,0,7538,1,0,0,0,1,1,1,0
6,898,3,1,1,330972,0,0,2,0,1,2,1,0
7,899,2,0,1,248738,2,0,0,0,0,1,0,0
8,900,3,1,1,2657,0,0,1,0,1,3,1,0
9,901,3,0,1,A/4 48871,2,0,0,0,0,1,0,0


In [111]:

del train_dataset['Cabin']
del test_dataset['Cabin']

# Delete Ticket column from datasets  (No need for them in the analysis)
del train_dataset['Ticket']
del test_dataset['Ticket']

del train_dataset['Port']
del test_dataset['Port']

dataset.head(20)

,PassengerId,Pclass,Sex,Age,Fare,Has_Cabin,IsAlone,Title,FamilySizeGroup,IsChildandRich
0,892,3,0,2,0,0,1,1,1,0
1,893,3,1,2,0,0,0,3,0,0
2,894,2,0,3,1,0,1,1,1,0
3,895,3,0,1,1,0,1,1,1,0
4,896,3,1,1,1,0,0,3,0,0
5,897,3,0,0,1,0,1,1,1,0
6,898,3,1,1,0,0,1,2,1,0
7,899,2,0,1,2,0,0,1,0,0
8,900,3,1,1,0,0,1,3,1,0
9,901,3,0,1,2,0,0,1,0,0


In [112]:
print("---------finish Data cleaning --------")

---------finish Data cleaning --------


In [113]:
del train_dataset['PassengerId']

In [114]:
train_dataset.head(20)

,Survived,Pclass,Sex,Age,Fare,Has_Cabin,IsAlone,Title,FamilySizeGroup,IsChildandRich
0,0,3,0,1,0,0,0,1,0,0
1,1,1,1,2,3,1,0,3,0,0
2,1,3,1,1,1,0,1,2,1,0
3,1,1,1,2,3,1,0,3,0,0
4,0,3,0,2,1,0,1,1,1,0
5,0,3,0,2,1,0,1,1,1,0
6,0,1,0,3,3,1,1,1,1,0
7,0,3,0,0,2,0,0,4,2,0
8,1,3,1,1,1,0,0,3,0,0
9,1,2,1,0,2,0,0,3,0,1


###### MODEL EVALUATION

In [115]:
X_train = train_dataset.drop("Survived",axis=1)
Y_train = train_dataset["Survived"]
X_test  = test_dataset.drop("PassengerId",axis=1).copy()

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(891, 9)
(891,)
(418, 9)


In [119]:
#Learn Curve
cv=ShuffleSplit(n_splits=100,test_size=0.2,random_state=0)
logreg_model=LogisticRegression()

def Learning_curve_model(X,Y,model,cv,train_sizes):
    plt.figure()
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(model, X, Y, cv=cv, n_jobs=4, train_sizes=train_sizes)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std  = np.std(train_scores, axis=1)
    test_scores_mean  = np.mean(test_scores, axis=1)
    test_scores_std   = np.std(test_scores, axis=1)
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
                     
    plt.legend(loc="best")
    return plt

#learn curve
if plot_lc==1:
    train_size=np.linspace(.1, 1.0, 15)
    Learning_curve_model(X_train,Y_train , logreg_model, cv, train_size)

In [120]:
# Logistic Regression
logreg = LogisticRegression() #(C=0.1, penalty='l1', tol=1e-6)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

result_train = logreg.score(X_train, Y_train)
result_val = cross_val_score(logreg,X_train, Y_train, cv=5).mean()
print('taring score = %s , while validation score = %s' %(result_train , result_val))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


taring score = 0.8159371492704826 , while validation score = 0.8103107433291947


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [121]:
submission = pd.DataFrame({
        "PassengerId": test_dataset["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('output.csv', index=False)
print('Exported')

Exported
